# Meter Simulator
This notebook keeps generating values for a power meter, and sends them through a RabbitMQ channel with a frequency.

In [ ]:
%reload_ext autoreload
%autoreload 2
from meter import Meter ## Meter class
import pika, time
from datetime import datetime as dt, timedelta
import shared_params as params, pandas as pd

### Setup Parameters
Set the meter parameters

In [ ]:
random_seed = None        # random seed for reproducible results
meter_range = (0,9000)    # the range for the power meter.

## Meter Simulation
The simulation is realized here. Note that, simulation is done real time, where 1 day of simulation takes 1 day to complete. For a quick simulation, see section [Quick Simulation](#quick_simulation).


### Start Meter

Initialize Meter

In [ ]:
meter = Meter( 
    random_seed=random_seed,
    meter_range=meter_range
)
start_time = dt.now()
## send file name to be saved to the 
file_name=str(start_time.year)+"-"+str(start_time.month)+"-"\
        +str(start_time.day)+"-"+str(start_time.hour) +"-"+str(start_time.minute)+".csv"
meter.send_init(
    file_name=file_name
)
print("File name to be saved:",file_name)

### Start Simulation
Determine the time frame to run, and start the reading and sending the values of the meter.

In [ ]:
now = start_time = dt.now()
until = now +timedelta(minutes=1,days=0)

while now < until:
    meter.process_meter(now.strftime(params.datetime_format),0)
    now = dt.now()
    time.sleep(1.0 - (now - start_time).seconds % 1.0)
meter.process_meter(now.strftime(params.datetime_format),1)

## Quick Simulation
<a id="quick_simulation"></a>
In this part, the simulation of one day is realized quickly, unlike the part above where you have to wait for the whole duration of simulation.
### Initialize Meter

In [ ]:
meter = Meter( 
    random_seed=random_seed,
    meter_range=meter_range
)
start_time = dt.now()
## send file name to be saved to the 
file_name=str(start_time.year)+"-"+str(start_time.month)+"-"\
        +str(start_time.day)+"-"+str(start_time.hour) +"-"+str(start_time.minute)+".csv"
meter.send_init(
    file_name=file_name
)
print("File name to be saved:",file_name)

### Start Quick Simulation
Determine the range of simulation first.

In [ ]:
now = dt(2019,12,13,0,0,0)
tomorrow = now +timedelta(hours=0, minutes=0,days=1)

Create the timestamps for the application, the simulation is done in frequency of seconds

In [ ]:
index = pd.date_range(start=now,end=tomorrow, freq="S")

Start it...

In [ ]:
for ts in index:
    meter.process_meter(ts.strftime(params.datetime_format),0)
meter.process_meter(dt.now().strftime(params.datetime_format),1)